In [2]:
import pandas as pd
import io
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
print ('Ran')

Ran


In [3]:
app_train = pd.read_csv('D:/Project_Data/application_train.csv')
app_test = pd.read_csv('D:/Project_Data/application_test.csv')

print("Shape of training data: {}".format(app_train.shape))
print("Shape of test data: {}".format(app_test.shape))

Shape of training data: (307511, 122)
Shape of test data: (48744, 121)


In [4]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
train_labels = app_train['TARGET']
train_sk_id_curr = app_train['SK_ID_CURR']
test_sk_id_curr = app_test['SK_ID_CURR']

app_train.drop('SK_ID_CURR', inplace=True, axis=1)
app_test.drop('SK_ID_CURR', inplace=True, axis=1)

app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 120)
Testing Features shape:  (48744, 120)


In [6]:
cat_features = [f for f in app_train.columns if app_train[f].dtype == 'object']

In [7]:
def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, query_cols, sorter=sidx)]

In [8]:
cat_features_inds = column_index(app_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_inds)

Cat features are: ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']
[ 0  1  2  3  9 10 11 12 13 26 30 38 84 85 87 88]


In [9]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in cat_features:
    app_train[col] = le.fit_transform(app_train[col].astype(str))
    app_test[col] = le.fit_transform(app_test[col].astype(str))

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1

print('%d columns were label encoded.' % le_count)

app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

In [10]:
print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 120)
Testing Features shape:  (48744, 120)


In [13]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')
imputer.fit(app_train)
app_train = imputer.transform(app_train)
app_test = imputer.transform(app_test)

In [14]:
from sklearn.model_selection import train_test_split, cross_val_score
import xgboost as xgb

In [15]:
ratio = (train_labels == 0).sum()/ (train_labels == 1).sum()
ratio

11.387150050352467

0 - Non-defaulter and 1- Defaulter

In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score

In [17]:
X_train, X_test, y_train, y_test = train_test_split(app_train, train_labels, test_size=0.2, stratify=train_labels, random_state=1)
print("Postive examples in train set: {}".format(np.sum(y_train==0)))
print("Negative examples in train set: {}".format(np.sum(y_train==1)))

print("Postive examples in test set: {}".format(np.sum(y_test==0)))
print("Negative examples in test set: {}".format(np.sum(y_test==1)))

Postive examples in train set: 226148
Negative examples in train set: 19860
Postive examples in test set: 56538
Negative examples in test set: 4965


In [18]:
clf = XGBClassifier(n_estimators=1000, objective='binary:logistic', gamma=0.1, subsample=0.5, scale_pos_weight=ratio )
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='auc', early_stopping_rounds=10)

[0]	validation_0-auc:0.684694
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.69767
[2]	validation_0-auc:0.701373
[3]	validation_0-auc:0.705643
[4]	validation_0-auc:0.70868
[5]	validation_0-auc:0.710959
[6]	validation_0-auc:0.712839
[7]	validation_0-auc:0.714636
[8]	validation_0-auc:0.715176
[9]	validation_0-auc:0.717006
[10]	validation_0-auc:0.718398
[11]	validation_0-auc:0.720092
[12]	validation_0-auc:0.721173
[13]	validation_0-auc:0.724314
[14]	validation_0-auc:0.72539
[15]	validation_0-auc:0.726071
[16]	validation_0-auc:0.728094
[17]	validation_0-auc:0.728817
[18]	validation_0-auc:0.730411
[19]	validation_0-auc:0.730801
[20]	validation_0-auc:0.732192
[21]	validation_0-auc:0.733783
[22]	validation_0-auc:0.734385
[23]	validation_0-auc:0.734824
[24]	validation_0-auc:0.735173
[25]	validation_0-auc:0.735963
[26]	validation_0-auc:0.737605
[27]	validation_0-auc:0.738684
[28]	validation_0-auc:0.738828
[29]	validation_0-auc:0.739251
[30]	validation_0-a

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0.1,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=11.387150050352467,
       seed=None, silent=None, subsample=0.5, verbosity=1)

In [19]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)
print('Accuracy score ',accuracy_score(y_test, y_pred))

Accuracy score  0.7004698957774418


In [20]:
n_estimators = clf.best_ntree_limit
clf = XGBClassifier(n_estimators=n_estimators, objective='binary:logistic', gamma=0.1, subsample=0.5, scale_pos_weight=ratio )
clf.fit(app_train, train_labels, eval_set=[(app_train, train_labels)], eval_metric='auc')

[0]	validation_0-auc:0.686937
[1]	validation_0-auc:0.691054
[2]	validation_0-auc:0.706216
[3]	validation_0-auc:0.708388
[4]	validation_0-auc:0.708699
[5]	validation_0-auc:0.713207
[6]	validation_0-auc:0.714295
[7]	validation_0-auc:0.716832
[8]	validation_0-auc:0.717939
[9]	validation_0-auc:0.719787
[10]	validation_0-auc:0.720925
[11]	validation_0-auc:0.72301
[12]	validation_0-auc:0.723734
[13]	validation_0-auc:0.726398
[14]	validation_0-auc:0.727451
[15]	validation_0-auc:0.728416
[16]	validation_0-auc:0.728998
[17]	validation_0-auc:0.729529
[18]	validation_0-auc:0.73024
[19]	validation_0-auc:0.731471
[20]	validation_0-auc:0.73262
[21]	validation_0-auc:0.733088
[22]	validation_0-auc:0.734061
[23]	validation_0-auc:0.735106
[24]	validation_0-auc:0.73597
[25]	validation_0-auc:0.736875
[26]	validation_0-auc:0.738188
[27]	validation_0-auc:0.738648
[28]	validation_0-auc:0.739556
[29]	validation_0-auc:0.740707
[30]	validation_0-auc:0.741544
[31]	validation_0-auc:0.741986
[32]	validation_0-auc:

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0.1,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=241, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=11.387150050352467,
       seed=None, silent=None, subsample=0.5, verbosity=1)

In [21]:
predictions = clf.predict(app_test)

In [22]:
submission = pd.DataFrame({'SK_ID_CURR': test_sk_id_curr, 'TARGET': predictions})

In [23]:
submission.head()

,SK_ID_CURR,TARGET
0,100001,0
1,100005,1
2,100013,0
3,100028,0
4,100038,1


In [24]:
submission.to_csv('baseline_xgboost_trial_only_labelencoded.csv', index = False)